# Max-Cut (QuTiP)

This is a simple template showing how one could use QuTiP to evaluate
the Max-Cut cost Hamiltonian on computational basis states and time
a brute-force search on small instances.


In [ ]:
import time
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from qutip import tensor, qeye, sigmaz


In [ ]:
n_nodes = 5
G = nx.erdos_renyi_graph(n_nodes, 0.5, seed=1)
pos = nx.spring_layout(G, seed=1)
nx.draw(G, pos, with_labels=True)
plt.title("Max-Cut instance (QuTiP)")
plt.show()

n = len(G.nodes())


In [ ]:
def z_op(i, n):
    ops = [qeye(2)] * n
    ops[i] = sigmaz()
    return tensor(ops)

H = 0
for i, j in G.edges():
    H += z_op(i, n) * z_op(j, n)


In [ ]:
def energy_for_bitstring(bits):
    # bits is list of 0/1
    # Construct the computational basis state |bits>
    from qutip import basis
    k = 0
    for idx, b in enumerate(bits):
        k |= (b << idx)
    # k is integer label of the basis state, dimension 2^n
    dim = 2**n
    psi = basis(dim, k)
    E = (psi.dag() * H * psi).full().item()
    return np.real(E)


In [ ]:
start = time.perf_counter()

best_E = 1e9
best_bits = None

for state in range(2**n):
    bits = [(state >> i) & 1 for i in range(n)]
    E = energy_for_bitstring(bits)
    if E < best_E:
        best_E = E
        best_bits = bits

end = time.perf_counter()

print("Best energy:", best_E)
print("Best bitstring:", best_bits)
print(f"Runtime (brute force 2^{n} states): {end - start:.4f} s")
